# Topic Modelling with Latent Dirichlet Allocation on FOMC Meeting Minutes 
## Conan Tan

# Introduction

This is a little project that I put together to better understand topic modelling, specifically Latent Dirichlet Allocation (LDA), by analyzing minutes from 70 different FOMC meetings ranging from 2009-2018. Coming out of undergraduate, I wondered how data science and economics would blend together as more and more economists begin to use data science approaches. I by no means am an economist nor a data scientist, but this project helped me uncover how both these fields perfectly compliment one another. 

In [1]:
#Import libraries
%matplotlib inline
import requests, re, pyLDAvis.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from ipywidgets import widgets, interactive
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text #edit stop word list
from sklearn.model_selection import GridSearchCV #grid searchs to find optimal parameters for LDA
from collections import Counter 
from nltk.stem.snowball import EnglishStemmer

In [2]:
#Missing meetings 20140618, 20131218, 20100316, 20090128,20091104, # 20080130, 20080318, 20080430, 20080625, 20080805, 20080916, 20081029, 20081216] 
meetingDates = [  20180131, 20180321, 20180502, 
                  20170201, 20170315, 20170503, 20170614, 20170726, 20170920, 20171101, 20171213, 
                  20160127, 20160316, 20160427, 20160615, 20160727, 20160921, 20161102, 20161214, 
                  20150128, 20150318, 20150429, 20150617, 20150729, 20150917, 20151028, 20151216, 
                  20140129, 20140319, 20140430, 20140730, 20140917, 20141029, 20141217, 
                  20130130, 20130320, 20130501, 20130619, 20130731, 20130918, 20131030, 
                  20120125, 20120313, 20120425, 20120620, 20120801, 20120913, 20121024, 20121212,
                  20110126, 20110315, 20110427, 20110622, 20110809, 20110921, 20111102, 20111213,
                  20100127, 20100428, 20100623, 20100810, 20100921, 20101103, 20101214,
                  20090318, 20090429, 20090624, 20090812, 20090923, 20091216]


In [3]:
#Recursively add tag's string to returnable string
def recursiveTagSearch(text, tag, last):  
  #skip if string is None
  if tag.string == None:
    return recursiveTagSearch(text, tag.next, last)
  if tag.string != last:
    text += tag.string
  if tag.next.string == 'Voting for this action:':
    return text
  else:
    return recursiveTagSearch(text, tag.next, tag.string)

In [4]:
#Scrapes Fed website for FOMC meeting minutes
def FOMCscrapper():
  baseURL = 'https://www.federalreserve.gov/monetarypolicy/fomcminutes'
  minutes = [] 
  #Iterate through FOMC Meeting Dates and append selected text to list
  for date in meetingDates:
    currentURl = baseURL + str(date) + '.htm'
    page = requests.get(currentURl)
    soup = BeautifulSoup(page.content, 'html.parser')
    print('Scrapping meeting date:', date)
    minutes.append(recursiveTagSearch('',soup.find('strong', text = re.compile('^Developments')), ''))
  return minutes

# Data Gathering and Preprocessing

We begin building our corpus (a collection of text) by scraping the Federal Reserve website for meeting minutes from the years 2009-2018. I was only able to gather 70 meeting minutes since some meetings had special HTML tags that were not compatabile with my scraper. After fetching the data, we begin the preprocessing phase by stemming the text. Stemming is the process of taking a word, or token, and reducing it into its root word. For example, any plurals or possessives such as "Dogs" or "Powell's" is changed to "Dog" and "Powell". Next, we remove any stop words from our data. Stop words are common words that are likely to appear frequently in our data and add no insightful value. It seems that the FOMC have a propensity to reference their time frames in months. Let's remove the names of months from the data since they do not seem to add value. Remember the data has already been stemmed so to remove the months, we must specify the stemmed name of the months. 

In [5]:
#Load documents into array
minutes = FOMCscrapper()

Scrapping meeting date: 20180131
Scrapping meeting date: 20180321
Scrapping meeting date: 20180502
Scrapping meeting date: 20170201
Scrapping meeting date: 20170315
Scrapping meeting date: 20170503
Scrapping meeting date: 20170614
Scrapping meeting date: 20170726
Scrapping meeting date: 20170920
Scrapping meeting date: 20171101
Scrapping meeting date: 20171213
Scrapping meeting date: 20160127
Scrapping meeting date: 20160316
Scrapping meeting date: 20160427
Scrapping meeting date: 20160615
Scrapping meeting date: 20160727
Scrapping meeting date: 20160921
Scrapping meeting date: 20161102
Scrapping meeting date: 20161214
Scrapping meeting date: 20150128
Scrapping meeting date: 20150318
Scrapping meeting date: 20150429
Scrapping meeting date: 20150617
Scrapping meeting date: 20150729
Scrapping meeting date: 20150917
Scrapping meeting date: 20151028
Scrapping meeting date: 20151216
Scrapping meeting date: 20140129
Scrapping meeting date: 20140319
Scrapping meeting date: 20140430
Scrapping 

In [7]:
#Stem each word in doc
stemmer = EnglishStemmer()
analyzer = CountVectorizer().build_analyzer()

for i in range(len(minutes)):
    tokenList = analyzer(minutes[i])
    for j in range(len(tokenList)):
        tokenList[j] = stemmer.stem(tokenList[j])
    minutes[i] = ' '.join(tokenList)

#Adjust stopwords as preferred, testing without months. 
stopWords = ['januari', 'februari', 'march', 'april', 'june', 'juli', 'septemb', 'august','novemb', 'octob', 'decemb']
stopWords = text.ENGLISH_STOP_WORDS.union(stopWords)

# Creating the Document Term Matrix and the LDA Model

Our data is currently charactized as a list of 70 items where each item is a string containing the entire meeting's minutes. Before we can use LDA, we must create a document term matrix (DTM). A document term matrix describes the frequency of words in a collection of documents. Intuitively, you can think of a DTM as a two-diminensional list of N documents where each document is represented by another list M words (every word in the collection) long that keeps count of the frequency of that word. Now we can finally use Latent Dirichlet Allocation to topic model our collection of data! 

# A Brief Introduction to Latent Dirichlet Allocation

LDA is a topic modelling approach that utilizes a bag of words approach. It states that any corpus (a collection of text) is comprised of K number of topics. Each topic then consists of a set of words that come directly from the documents. After the topics and the topic words are identified, LDA will then assign two probabilites: the probaility that a document belongs in a specific topic and the probability that a word belongs that topic. This, of course, is a surface level view of LDA. For more indepth surface level information, check out http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/ and for the statistically savvy https://ai.stanford.edu/~ang/papers/jair03-lda.pdf.

After some mathmatical wizandry courtesy of David Blei et al (2003), LDA took our 70 documents which consisted of 2320 unique words and created 5 topic groups (a number predetermined by me). Let's now analyze our documents and our topics!

In [8]:
#Create document term matrix using count vectorizer for LDA
countVectorizer = CountVectorizer(max_df = .5, stop_words = stopWords, lowercase = True)
countVectorized = countVectorizer.fit_transform(minutes)

#Create document term matrix using Term Frequency - Inverse Document Frequency
tfidfVectorizer = TfidfVectorizer(max_df = .5, stop_words = stopWords, lowercase = True)
tfidfVectorized = tfidfVectorizer.fit_transform(minutes)

In [9]:
#Create LDA model
topicNum = 5

ldaModel = LatentDirichletAllocation(n_components = topicNum, learning_method = 'batch', learning_decay = .9, random_state = 94)
ldaModelFit = ldaModel.fit_transform(countVectorized)
print('Document Term Matrix Shape: (N docs, M Words)', countVectorized.shape, '\n')
print('LDA Vector Shape(N Docs, K Topics):', ldaModelFit.shape)

Document Term Matrix Shape: (N docs, M Words) (70, 2320) 

LDA Vector Shape(N Docs, K Topics): (70, 5)


In [10]:
#Find topic distributions for each document per topic
#topicDist = [doc.tolist() for doc in ldaModelFit]

#Make meeting dates suitable for printing
printDates = []
for i in range(len(meetingDates)):
    printDates.append('Document ' + str(i) + ' - ' + str(meetingDates[i]))
    
#Create interactive widgets for visualization
documents = widgets.Dropdown(
    options = printDates,
    value = printDates[0],
    description = 'Documents'
)
topWords = widgets.IntSlider(
    value = 15,
    min = 0,
    max = 30,
    description = "N Words",
    readout_format = 'd'
)

# Word Frequency Counter, TF-IDF Score, and Topic Distribution

Here we have a small interactive visualization that allows you to choose a document to view its word frequency distribution, TF-IDF score, as well as its topic distribution. The Term Frequency - Inverse Document Frequency (TF-IDF) is a metric to measure how important a term is to a document. Higher scores indicate the term is rarer to the given document and more emphasis should be placed during analysis. Comparisons between word frequency and TF-IDF showcase how some words might not be as important as its simple frequency depicts. Topic distribution refers to the probability that a document falls into the selected topic. 




From this visualization, the basic topics that are talked about during the meeting as well as how much of an emphasis is put on that topic. It appears that some meetings have word frequency highs in the 30's while some only peak below 10. Looking specifically at document 8 with meeting date 20170920, the meeting immediately following the severe hurricane season of 2017, we can see that the main topic of the FOMC meeting was in fact the storms. More on this document later.

In [11]:
#Get word frequencies for specified docs
def getTopWords(vectorizer, vectorized, docNum, n):
    featureNames = list(vectorizer.get_feature_names())
    counts = vectorized[docNum].sum(axis=0).A1
    freqDistribution = Counter(dict(zip(featureNames, counts))).most_common(n)
    xValues = [val[0] for val in freqDistribution]
    yValues = [val[1] for val in freqDistribution]
    xValues.reverse()
    yValues.reverse()
    return [xValues, yValues]

In [12]:
#Print bar chart showing top N words of a document
def plotFreqDist(topWords, documents):
    
    docNum = int(documents.split()[1])
    
    #Adjust error where pyLDAvis and sklearn LDA have different topic orderings
    topicsReorder = [ldaModelFit[docNum][1], ldaModelFit[docNum][3], ldaModelFit[docNum][2],
                     ldaModelFit[docNum][4], ldaModelFit[docNum][0]]
    
    #Create plot
    plt.figure(figsize =(14,10))
    wordFreqDist = plt.subplot(2,2,1)
    tfidfDist = plt.subplot(2,2,2)
    topicDist = plt.subplot(2,2,3)

    countVect = getTopWords(countVectorizer, countVectorized, docNum, topWords)
    tfidfVect = getTopWords(tfidfVectorizer, tfidfVectorized, docNum, topWords)
    wordFreqDist.barh(countVect[0], countVect[1], color = 'y')
    tfidfDist.barh(tfidfVect[0], tfidfVect[1], color = 'm')
    topicDist.bar([1,2,3,4,5],topicsReorder, color = 'c')
    
    title = 'Document '+ str(docNum)
    ylabel = 'Top ' + str(topWords) + ' Terms'
    plt.suptitle(title)
    wordFreqDist.set_title('Word Frequency')
    wordFreqDist.set_ylabel(ylabel)
    wordFreqDist.set_xlabel('Frequency')
    tfidfDist.set_title('TF-IDF Score')
    tfidfDist.set_xlabel('p')
    topicDist.set_title('Topic Probability')
    topicDist.set_ylabel('Prob')
    topicDist.set_xlabel('Topics')
    plt.show()

interactive(plotFreqDist, topWords = topWords, documents = documents)

A Jupyter Widget

# The Final LDA Visualization

Finally, we have a visualization that sums up the entire LDA model. On the left, we have a intertopic distance map. The size of the topic refers to its overall weight in the model and the distances show how related the topics are to one another. On the right, we have a count of the words that appear in the document. Clicking on a topic shows the words that most contribute to that topic. Grey indicates the total count of that word in the entire corpus, while the red shading shows the proportion of that count that was used in the specified topic. The lamda slider sorts the words based its relative weighting to that topic. A lower value shows words that more uniquely correlate with that topic, while a value of 1 sorts by word frequency in that topic. Let's begin some analysis into the topics and the words that comprise our topics! 


In [13]:
#Create pyLDA visualization 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(ldaModel, countVectorized, countVectorizer, mds = 'tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      32.825051        1       1   68.288986 -264.695374
3      24.850232        1       2  148.827652   33.405369
2      24.541146        1       3  233.056366 -193.526459
4      14.836226        1       4   16.013798  -86.535347
0       2.947345        1       5  320.678406  -36.897682, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
998   Default   59.000000     hurrican   59.000000  30.0000  30.0000
1762  Default  157.000000          rrp  157.000000  29.0000  29.0000
53    Default   88.000000         2017   88.000000  28.0000  28.0000
45    Default  105.000000         2010  105.000000  27.0000  27.0000
669   Default   88.000000      disrupt   88.000000  26.0000  26.0000
910   Default   81.000000      gasolin   81.000000  25.0000  25.0000
742   Default  102.000000          eme  102.000000  24.0000  24.0000
44    Default   65.000000         2009   65.000000  23.0000  23.0000
52    Default  102.000000         2016  102.000000  22.0000  22.0000
944   Default  163.000000      guidanc  163.000000  21.0000  21.0000
150   Default   64.000000          afe   64.000000  20.0000  20.0000
2161  Default   86.000000    underutil   86.000000  19.0000  19.0000
532   Default   91.000000          cpi   91.000000  18.0000  18.0000
54    Default   53.000000         2018   53.000000  17.0000  17.0000
170   Default   53.000000           al   53.000000  16.0000  16.0000
47    Default   98.000000         2012   98.000000  15.0000  15.0000
46    Default  105.000000         2011  105.000000  14.0000  14.0000
2030  Default   31.000000         talf   31.000000  13.0000  13.0000
50    Default  102.000000         2015  102.000000  12.0000  12.0000
1891  Default   80.000000      softwar   80.000000  11.0000  11.0000
1204  Default   57.000000         loss   57.000000  10.0000  10.0000
1427  Default   75.000000        paper   75.000000   9.0000   9.0000
2057  Default   78.000000         test   78.000000   8.0000   8.0000
2018  Default   32.000000      symmetr   32.000000   7.0000   7.0000
55    Default   39.000000         2019   39.000000   6.0000   6.0000
2183  Default   46.000000        unlik   46.000000   5.0000   5.0000
1179  Default   39.000000      liftoff   39.000000   4.0000   4.0000
43    Default   41.000000         2008   41.000000   3.0000   3.0000
1950  Default   59.000000     stimulus   59.000000   2.0000   2.0000
1318  Default   42.000000      neutral   42.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
975    Topic5    1.356387         henc    3.876488   2.4742  -6.3330
204    Topic5    4.064668        apart   22.949060   1.7933  -5.2355
498    Topic5    1.355972      contour    3.920855   2.4625  -6.3333
916    Topic5    3.569133     geopolit   18.709539   1.8676  -5.3655
53     Topic5    8.973543         2017   88.596285   1.2345  -4.4436
179    Topic5    3.765500        alter   22.350682   1.7433  -5.3120
1068   Topic5    1.649355        innov    5.678801   2.2879  -6.1375
669    Topic5    7.938926      disrupt   88.360210   1.1146  -4.5661
1514   Topic5    1.145330       postur    3.168448   2.5067  -6.5022
1973   Topic5    2.589184  subcommitte   13.617316   1.8643  -5.6865
910    Topic5    6.916413      gasolin   81.518362   1.0573  -4.7040
260    Topic5    2.509684        await   13.653618   1.8304  -5.7177
527    Topic5    4.117423        cours   39.885105   1.2535  -5.2226
1105   Topic5    2.034431           ip    9.033452   2.0335  -5.9276
2046   Topic5    3.708280  temporarili   36.305500   1.2429  -5.3273
2183   Topic5    3.972567        unlik   46.764831   1.0585  -5.2584
742    Topic5    5.425242          eme  102.962353   0.5810  -4.9468
128    Topic5    3.203178   acknowledg   31.185167   1.2485  -5.4737
835    Topic5    3.426433       


# Analysis

The topics are very spread apart from one another with no overlaps, indicating that each topic is fairly unique. Upon inspection of each topic, it is clear that the topics are grouped by timeframes. If the years are not a big enough give away enough for you, let's examine topic 4. Set lambda to 0.6, which is thought to provide the most interpretable words for discerning topics from one another. The first 5 words are "2009", "2010, "talf", "loss", and "2008". The most prominent thing to me at first was "talf", mainly because I have no idea what that word means. Term Asset Backed Securities Loan Facility (TALF) is a Federal Reserve stimulus program created in 2008 that lasted until 2010 that promoted consumer spending through the issuance of asset backed securities. Going through the rest of the topics, you can find all sorts of little connections like that between the words.

Topic 5 provides the most unique bag of words. It is the smallest circle, showing that it is the least weighted topic. However, every word relates to the tropical storms that occured in 2017. While this topic only captured one specific document, it shows that particular meeting/document was unique enough to command its own entire topic. If you continue to inspect the topics and the document topic distributions above, you will continue to see trends in groupings based on time. 

# Limitations and Future Expansion

However, the largest limitation faced in this project was in fact the corpus itself. After removal of words that occured in X number of documents, the unique word count dropped dramatically. The meeting minutes shared a lot of similarity in both formatting and in word structure. While that provided ease in web scraping, it limited the overall model strength. 

This was only my first foray into unsupervised machine learning models and was a great learning exercise. There are several areas that I wish to expand on in the near future. The corpus should be expanded to capture FOMC meetings prior to the recession to gather a more holistic view of the evolution of the meetings post crisis. The preprocessing phase as well as tuning and experimentation with the LDA hyperparameters could  be refined for better more interpretable results. Removal of years from the token list and then having a human attempt to classify topic years  may prove to be a strong cross validation of the model. I will hopefully expand on this particular project in the future, if not I will atleast take everything I learned and apply it to the next one.  